Boat Recognition

In this project we will use Boat dataset from kaggle: https://www.kaggel.com/clorichel/boat-types-recognition 

in order to download the dataset use command " !kaggle datasets download -d clorichelt/boat-types-recognition -p./data"

In [ ]:
!kaggle datasets download -d clorichel/boat-types-recognition -p ./data

In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn functional as F 
import torch.backends.cudnn as cudnn 

import torchvision 
import torchvision.transforms as transforms 

import matplotlib.pyplot as plt 
import numpy as np 

from tqdm import tqdm 



In [ ]:
ls ./data

In [ ]:
transform = transform.Compose([
    transforms.Resize([224,224]), #resizing the image,
    transforms.RandomRotation(degrees=[-30, 30]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomGrayscale(),
    transforms.ToTensor(), #convert images to pytorch tensor,
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #Normalization
])


In [ ]:
dataset = torchvision.datasets.ImageFolder('./data/', transform=transform)

In [ ]:
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size 
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


trainloader = torch.utls.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
len(dataset)

In [ ]:
dataset.classes

In [ ]:
def imshow(img):
    plt.figure(figsize=(10, 10))
    img = img / 2 + 0.5 #unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

In [ ]:
net = torchvision.models.resnet18(pretrained=True)

In [ ]:
net

In [ ]:
for param in net.parameters():
    param.requires_grad = False

In [ ]:
net.fc = nn.Linear(512, 9)

In [ ]:
net

In [ ]:
net = net.cuda()

In [ ]:
net(images.cuda()).shape

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [ ]:
def get_quality(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader: 
            images, labels = data 
            images, labels = images.cuda(), labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print('Accuracy of the network on test images: %d %%' % (100 * correct / total))

def train_network(net, n_epochs=10):
    losses = []

    for epoch in range(n_epochs): # loop over the dataset mulitple times

        running_loss = 0.0
        for i, data in enumerate(tqdm(trainloader)):

            # get the inputs
            images, labels = data
            images, labels = images.cuda(), labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward pass + backward propagation + optimize
            outputs = net(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.items()
            losses.append(loss.item())
       
        print(' [%d, %5d] loss: %.3f' %
             (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

        get_quality(net)

plt.plot(losses, 'b')
plt.xlabel('Iteration')
plt.ylabel('Loss')

print('Finished Training')
